In [1]:
import pandas as pd
import numpy as np
import os

directory='E:\\CUHK(SZ)\\courses\\22-23 Term2\\DDA3020\\Assignments\\Assignment4'
os.chdir(directory)

In [2]:
df=pd.read_csv('seeds_dataset.txt',sep='\t',header=None,names=['col1','col2','col3','col4','col5','col6','col7','col8'])
true_label=df.col8
df=df.drop(['col8'],1)

df

,col1,col2,col3,col4,col5,col6,col7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175
...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044


# PCA

In [3]:
cov=df.cov()

eigen=np.linalg.eig(cov)
eigen_value=eigen[0]
eigen_vector=eigen[1]

eigen_value #first and second are the largest two eigenvalues

array([1.07933269e+01, 2.12945512e+00, 7.36300330e-02, 1.28874947e-02,
       2.74822668e-03, 1.57044980e-03, 2.96554425e-05])

In [4]:
pca=eigen_vector[:,0:2]
proj=df@pca
proj=pd.DataFrame(proj)
proj.columns=['col1','col2']

proj

,col1,col2
0,-20.863299,5.185150
1,-20.515517,3.913242
2,-19.539351,5.470965
3,-19.144575,4.981281
4,-21.819850,4.419086
...,...,...
205,-17.157978,6.129345
206,-16.090788,6.711812
207,-17.699817,10.910436
208,-16.867772,6.077181


# K-means

In [5]:
def cluster(dataset,K,max_iter=200):
    
    centroid=dataset.iloc[:K,:2]
    dataset['label']=np.nan
    
    for iteration in range(max_iter):
        
        # assigning each points to the closest clustering centre
        for ind in range(len(dataset)): 
            distance=np.linalg.norm(centroid-dataset.iloc[ind,:2],axis=1)
            index=np.where(distance==min(distance))[0][0]
            proj.iloc[ind,2]=index
        
        # recomputing the clutering centres
        centroid_new=proj.groupby('label').aggregate({'col1':'mean','col2':'mean'}).reset_index(drop=True)
        
        if (centroid_new==centroid).all().all():
            break
        else:
            centroid=centroid_new
    
    return centroid,dataset

In [6]:
result=cluster(proj,3)[1]

result.loc[result.label==2,'label']=3
result.loc[result.label==0,'label']=2

result

,col1,col2,label
0,-20.863299,5.185150,1.0
1,-20.515517,3.913242,1.0
2,-19.539351,5.470965,1.0
3,-19.144575,4.981281,1.0
4,-21.819850,4.419086,1.0
...,...,...,...
205,-17.157978,6.129345,3.0
206,-16.090788,6.711812,3.0
207,-17.699817,10.910436,3.0
208,-16.867772,6.077181,3.0


# Silhouette

In [7]:
def silhouette(dataset):
    s_list=[]
    for i in range(len(dataset)):
        dot=dataset.iloc[i]
        rest=dataset.drop(i)
        label=dot.label
        
        rest['distance']=np.sqrt((rest.col1-dot.col1)**2+(rest.col2-dot.col2)**2)
        
        # the average distance to the points in each cluster
        silhouette=rest.groupby('label').aggregate({'distance':'mean'}).reset_index() 
        
        a=silhouette.loc[silhouette.label==label,'distance'].min()
        b=silhouette.loc[silhouette.label!=label,'distance'].min()

        s=(b-a)/max(a,b)
        s_list.append(s)
        
    return sum(s_list)/len(s_list)

In [8]:
silhouette(result)

0.4765771593987351

# Rand Index

In [9]:
result.columns=['co1','col2','label_train']
result['label_truth']=true_label

result

,co1,col2,label_train,label_truth
0,-20.863299,5.185150,1.0,1
1,-20.515517,3.913242,1.0,1
2,-19.539351,5.470965,1.0,1
3,-19.144575,4.981281,1.0,1
4,-21.819850,4.419086,1.0,1
...,...,...,...,...
205,-17.157978,6.129345,3.0,3
206,-16.090788,6.711812,3.0,3
207,-17.699817,10.910436,3.0,3
208,-16.867772,6.077181,3.0,3


In [10]:
def RandIndex(dataset):
    a=0; b=0
    n=len(dataset)
    
    for i in range(n):
        dot=dataset.iloc[i,2:]
        rest=dataset.iloc[(i+1):,2:]
        label1=dot.label_train
        label2=dot.label_truth
        
        same=rest.loc[rest.label_train==label1]
        diff=rest.loc[rest.label_train!=label1]

        a+=(same.label_truth==label2).sum()
        b+=(diff.label_truth!=label2).sum()
        
    return (a+b)/(n*(n-1)/2)

In [11]:
RandIndex(result)

0.8713602187286398

In [12]:
test=(result.label_train==result.label_truth)

sum(test)/210

0.8904761904761904